In [12]:
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt

In [13]:
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
from os import listdir

In [14]:
damagepath = "/code/Project2/COE379L-Project2/damage"
nodamagepath = "/code/Project2/COE379L-Project2/no_damage"

X = []
y = []

# Load damaged images and convert to black and white
for img_name in os.listdir(damagepath):
    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(damagepath, img_name)
        img = load_img(img_path)
        img_gray = img.convert('L')
        img_array = img_to_array(img)
        X.append(img_array)
        y.append(1)

# Load non-damaged images
for img_name in os.listdir(nodamagepath):
    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(nodamagepath, img_name)
        img = load_img(img_path)
        img_gray = img.convert('L')
        img_array = img_to_array(img)
        X.append(img_array)
        y.append(0) 

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

print("Loaded images:", X.shape)
print("Labels:", y.shape)

Loaded images: (21322, 128, 128, 3)
Labels: (21322,)


In [15]:
print("Total images:", X.shape)
print("Total labels:", y.shape)

Total images: (21322, 128, 128, 3)
Total labels: (21322,)


In [16]:
print("Image shape (Height x Width x Channels):", X[0].shape)
print("All image shapes unique:", np.unique([img.shape for img in X], axis=0))

Image shape (Height x Width x Channels): (128, 128, 3)
All image shapes unique: [[128 128   3]]


In [17]:
print("Min pixel value:", X.min())
print("Max pixel value:", X.max())

Min pixel value: 0.0
Max pixel value: 255.0


In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 43, stratify = y)